In [2]:
!nvidia-smi

Fri May 10 20:38:20 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PG5...  On   | 00000000:10:00.0 Off |                    0 |
| N/A   35C    P0    80W / 330W |   8857MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PG5...  On   | 00000000:13:00.0 Off |                    0 |
| N/A   

In [3]:
import sys
sys.path.insert(0, '/home/bhavul/bhavul/NEKO/')

In [4]:
from __future__ import annotations
# supports dataset in huggingface datasets library for now

import torch
import time
import os

import wandb
import numpy as np
import torch
import random
import os
from datetime import datetime

import wandb
import torch

from peft import LoraConfig, TaskType, get_peft_model
from accelerate import Accelerator
from accelerate import DistributedDataParallelKwargs
from accelerate import DataLoaderConfiguration

from datasets import load_dataset, concatenate_datasets
import numpy as np
from torch import nn
from typing import TYPE_CHECKING, List,Dict
from transformers import AutoTokenizer
import torch
import copy

from __future__ import annotations
from typing import Optional, Union, TYPE_CHECKING
import torch
import torch.nn as nn

import transformers
from transformers import AutoTokenizer

# import gato
from gato.transformers import GPT2Model
from gato.training.trainer import Trainer
from gato.training.schedulers import get_linear_warmup_cosine_decay_scheduler
from gato.tasks.task import Task
from gato.utils.utils import save_model
from gato.training.arguments import TrainingArgs


/home/bhavul/.conda/envs/neko/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
class GatoPolicy(nn.Module):
    def __init__(
        self,
        device: Union[torch.device, str],
        embed_dim: int,
        layers: int,
        heads: int,
        dropout: float,

        activation_fn='gelu',

        mu: int = 100,
        M: int = 256,

        patch_size: int = 16,
        resid_mid_channels: int = 132,
        num_groups: int = 32,
        position_vocab_size: int = 128,
        continuous_tokens: int = 1024,
        discrete_tokens: int = 1024,

        context_len=1024,

        use_pos_encoding: bool = True,
        use_patch_pos_encoding: bool = True,

        pretrained_lm: Optional[str] = None, # Optional, name of pretrained language model to use
        flash: bool = False, # TODO verify correctness
        tokenizer_model_name: str = 'gpt2',
        pad_seq: bool = False
    ):
        super().__init__()
        self.device = device
        self.context_len = context_len
        
        # Text Tokenizer
        self.text_tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name)        
        # tokens
        self.vocab_size = self.text_tokenizer.vocab_size 
        if self.text_tokenizer.pad_token is None:
            self.text_tokenizer.pad_token = self.text_tokenizer.eos_token
        

        if pretrained_lm is not None:
            print(f'loading pretrained GPT2 weights')
            config = transformers.GPT2Config.from_pretrained(pretrained_lm)
            config.attn_pdrop = dropout # 0.1
            config.resid_pdrop = dropout
            self.transformer = GPT2Model.from_pretrained(
                pretrained_lm,
                config=config,
            )
            embed_dim = config.n_embd
            assert self.transformer.wte.weight.shape[0] == self.text_tokens, "pretrained token/expected mimsatch" # potentially make text_tokens dynamic
        else:
            gate = False
            if activation_fn == 'geglu':
                gate = True
                activation_fn = 'gelu'
            config = transformers.GPT2Config(
                vocab_size=1,  # doesn't matter -- we don't use the vocab
                n_embd=embed_dim,
                n_head=heads,
                n_layer=layers,
                resid_pdrop=dropout,
                attn_pdrop=dropout,
                n_positions=context_len,
                n_inner=embed_dim * 4,
                activation_function=activation_fn,
            )
            config.n_ctx = context_len
            config.gate = gate
            config.flash = flash
            self.transformer = self.transformer = GPT2Model(config)
        
        # embedding tokens
        self.embed_token = nn.Embedding(self.vocab_size, embed_dim)
        if pretrained_lm is not None:
            self.embed_token.weight.data[:] = self.transformer.wte.weight.data
        
        
        # head
        self.predict_token = nn.Linear(embed_dim, self.vocab_size, bias=False)
        self.separator_token = nn.Parameter(torch.zeros(embed_dim))

    @property
    def module(self):
        return self

    def forward(self, inputs: Optional[list]=None, compute_loss=False, **kwargs):
        # tokenize inputs
        if inputs is not None:
            token_embeddings, tokens, token_masks, target_tokens, target_masks = self.tokenize_input_dicts(inputs)
        else:
            token_embeddings = kwargs['token_embeddings']
            tokens = kwargs['tokens']
            token_target_masks = kwargs['token_target_masks']
            token_masks = kwargs['token_masks']

        assert token_embeddings is not None, "token_embeddings is None"
        assert token_masks is not None, "token_masks is None"

        final_representations = self.transformer(inputs_embeds=token_embeddings, attention_mask=token_masks)['last_hidden_state']
        logits = self.predict_token(final_representations)
        # assert 'target' in kwargs, "target is not there in kwargs"

        # print(f"Type of target_tokens: {type(target_tokens)}")
        # print(f"Shape of target_tokens: {target_tokens.shape if isinstance(target_tokens, torch.Tensor) else 'N/A'}")
        # print(f"Type of pad_token_id: {type(self.text_tokenizer.pad_token_id)}")
        if compute_loss:
            # Ensuring target_tokens is a tensor
            if not isinstance(target_tokens, torch.Tensor):
                raise TypeError("target_tokens must be a torch.Tensor")
            
            # Correctly computing the loss mask
            loss_masks = (target_tokens != self.text_tokenizer.pad_token_id)
            if isinstance(loss_masks, torch.Tensor):
                loss_masks = loss_masks.float()  # Convert boolean tensor to float
            else:
                raise TypeError("Loss mask calculation did not return a tensor.")
            # loss_masks = (target_tokens != self.text_tokenizer.pad_token_id).float()
            loss = torch.nn.functional.cross_entropy(logits.view(-1, self.vocab_size), target_tokens.view(-1), reduction='none')
            loss = (loss * loss_masks.view(-1)).sum() / loss_masks.sum()
        else:
            loss = None
    
        return logits, loss


    def tokenize_input_dicts(self, inputs: list):
        # if not inputs:
        #     return None, None, None, None
    
        batch_len = len(inputs)
        max_input_tokens = max(len(batch['text']) for batch in inputs)
        max_target_tokens = max(len(batch['target']) for batch in inputs) if 'target' in inputs[0] else 0
        
        # Allocate tensors for input tokens
        token_embeddings = torch.zeros((batch_len, max_input_tokens, self.embed_token.embedding_dim), device=self.device)
        tokens = torch.zeros((batch_len, max_input_tokens), dtype=torch.long, device=self.device)
        token_masks = torch.zeros((batch_len, max_input_tokens), device=self.device)
        
        # Allocate tensors for target tokens if they exist
        target_tokens = torch.zeros((batch_len, max_target_tokens), dtype=torch.long, device=self.device)
        target_masks = torch.zeros((batch_len, max_target_tokens), device=self.device)
    
        for i, batch in enumerate(inputs):
            # Process input tokens
            input_tokens = batch['text'].to(device=self.device) if isinstance(batch['text'], torch.Tensor) else torch.tensor(batch['text'], dtype=torch.long, device=self.device)
            n_input_timesteps = len(input_tokens)
            
            tokens[i, :n_input_timesteps] = input_tokens
            token_embeddings[i, :n_input_timesteps] = self.embed_token(input_tokens)
            token_masks[i, :n_input_timesteps] = 1
            
            # Process target tokens if they exist
            if 'target' in batch:
                target_data = batch['target'].to(device=self.device) if isinstance(batch['target'], torch.Tensor) else torch.tensor(batch['target'], dtype=torch.long, device=self.device)
                n_target_timesteps = len(target_data)
                target_tokens[i, :n_target_timesteps] = target_data
                target_masks[i, :n_target_timesteps] = 1
    
        return token_embeddings, tokens, token_masks, target_tokens, target_masks


    def predict_text(self, batch_dict, max_length=20, deterministic=True, top_p=0.9):
        input_tokens = torch.tensor(batch_dict['text'], dtype=torch.long, device=self.device).unsqueeze(0)
        
        predicted_tokens = []
    
        for _ in range(max_length):
            token_embeddings = self.embed_token(input_tokens)
            token_masks = torch.ones_like(input_tokens)

            logits, _ = self.forward(token_embeddings=token_embeddings, tokens=input_tokens, token_target_masks=None, token_masks=token_masks)
            logits = logits[:, -1, :]  # focus on the last time-step logits
    
            if deterministic:
                token = torch.argmax(logits, dim=-1)
            else:
                probs = torch.nn.functional.softmax(logits, dim=-1)
                token = torch.multinomial(probs, 1)  # Sampling a token
    
            if token.numel() == 1:  # Checking if token is a single element
                predicted_tokens.append(token.item())
            else:
                print(f"Expected a single element, got {token.numel()} elements.")
    
            input_tokens = torch.cat([input_tokens, token], dim=1)  # Append the predicted token

            if token == self.text_tokenizer.eos_token_id:
                break
    
        return logits, predicted_tokens

In [31]:
class TextTask(Task): 
    def __init__(self, dataset_names:List[str], dataset_paths:List[str], context_length:int, tokenizer_model:str):
        super().__init__()
        self.context_length = context_length
        self.text_tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)
        if self.text_tokenizer.pad_token is None:
            self.text_tokenizer.pad_token = self.text_tokenizer.eos_token
        text_datasets_list = []
        assert len(dataset_names) == len(dataset_paths), "The dataset names and paths parameters should have corresponding values and hence equal lengths"
        for i, text_dataset in enumerate(dataset_names):
            text_datasets_list.append(load_dataset(path=dataset_paths[i], name=text_dataset))
        if len(text_datasets_list) == 1:
            self.text_dataset = text_datasets_list[0]
        else:            
            # https://huggingface.co/docs/datasets/v2.14.4/en/process#concatenate
            # must have the same feature columns
            self.text_dataset = concatenate_datasets(text_datasets_list)

    def sample_batch(self, batch_size, is_test=False) -> List[Dict]:
        split = 'train' if not is_test else 'test'
        dataset_split = self.text_dataset[split]

        if len(dataset_split) < batch_size:
            print(f"Warning: Requested batch size {batch_size} is larger than the dataset size {len(dataset_split)}.")
            batch_size = len(dataset_split)  # Adjust batch size to available data size

        if batch_size == 0:
            return []  # Early exit if no data is available

        
        sampled_indices = torch.randperm(len(dataset_split))[:batch_size]
        samples = dataset_split.select(sampled_indices)
        tokenized_outputs = self.text_tokenizer(samples['text'], truncation=True, padding="longest", max_length=self.context_length, return_tensors='pt')
    
        batch_dicts = []
        for input_ids in tokenized_outputs["input_ids"]:
            if input_ids.numel() > 0:  # Check if non-empty
                # Split into input and target tokens
                input_tokens = input_ids[:-1]
                target_tokens = input_ids[1:]
                batch_dicts.append({
                    'text': input_tokens,
                    'target': target_tokens,
                })
    
        return batch_dicts

    def evaluate(self, model: GatoPolicy, num_examples_to_test=50, deterministic=False, log_examples_to_output=False, is_test=True):
        split = 'train' if not is_test else 'test'
        dataset_split = self.text_dataset[split]
        num_examples_to_test = min(num_examples_to_test, len(dataset_split))
    
        if num_examples_to_test == 0:
            return {'loss': float('nan'), 'perplexity': float('nan')}
    
        batch_dicts = self.sample_batch(num_examples_to_test, is_test)
        total_loss, total_tokens = 0.0, 0
    
        for batch_dict in batch_dicts:
            input_tokens = batch_dict['text'].to(device=model.device)
            target_tokens = batch_dict['target'].to(device=model.device)
    
            total_loss_per_sequence = 0.0
            pred_tokens = []
    
            for idx in range(input_tokens.size(0)):
                pred_logits, single_pred_tokens = model.predict_text({'text': input_tokens[idx].unsqueeze(0)}, max_length=1, deterministic=deterministic)
                loss = torch.nn.functional.cross_entropy(pred_logits, target_tokens[idx].unsqueeze(0))
                total_loss_per_sequence += loss.item()
                pred_tokens.extend(single_pred_tokens)
            
            total_loss += total_loss_per_sequence / input_tokens.size(0)
            total_tokens += input_tokens.size(0)
    
            if log_examples_to_output:
                decoded_input = self.text_tokenizer.decode(input_tokens.squeeze(), skip_special_tokens=True)
                decoded_target = self.text_tokenizer.decode(target_tokens.squeeze(), skip_special_tokens=True)
                decoded_prediction = self.text_tokenizer.decode(torch.tensor(pred_tokens), skip_special_tokens=True)            
                print(f'=>Input: {decoded_input} \n =>Target: {decoded_target} \n =>Prediction: {decoded_prediction}')
    
        avg_loss = total_loss / total_tokens
        perplexity = torch.exp(torch.tensor(avg_loss)).item()
    
        return {'loss': avg_loss, 'perplexity': perplexity}

## trainer stuff

In [32]:
args = TrainingArgs(
    training_steps=10000,
    log_eval_freq=10,
    warmup_steps=500,
    batch_size=4,
    sequence_length=1024,
    eval_episodes=5,
    text_prop=1,
    # eval_text_log_examples=True,
    pretrained_lm=None,
    text_datasets=['wikitext-2-v1'],
    text_datasets_paths=['wikitext'],
    use_wandb=True,
    device='cuda',
    eval_mode='stochastic',
    eval_text_num_examples=5,
    # disable_cosine_decay=True
)

In [33]:
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
if args.use_wandb:
    log_with = 'wandb'
else:
    log_with = None
dl_config = DataLoaderConfiguration(split_batches=True)
accelerator = Accelerator(
    cpu=args.cpu,
    dataloader_config=dl_config, 
    # mixed_precision=args.mixed_precision,
    # gradient_accumulation_steps=args.gradient_accumulation_steps,
    kwargs_handlers=[ddp_kwargs],
    log_with=log_with
)
args.device = accelerator.device.type
exp_date = datetime.now().strftime('%y-%m-%d_%H-%M-%S')
exp_name = f'neko-gato_{exp_date}'

model = GatoPolicy(
        device=args.device,
        embed_dim=args.embed_dim,
        layers=args.layers,
        heads=args.heads,
        dropout=args.dropout,
        mu=args.mu,
        M=args.M,
        patch_size=args.patch_size,
        resid_mid_channels=args.resid_mid_channels,
        continuous_tokens=args.continuous_tokens,
        discrete_tokens=args.discrete_tokens,
        context_len=args.sequence_length,
        use_patch_pos_encoding=not args.disable_patch_pos_encoding,
        use_pos_encoding=not args.disable_inner_pos_encoding,
        activation_fn=args.activation_fn,
        pretrained_lm=args.pretrained_lm,
        flash=args.flash,
        tokenizer_model_name=args.tokenizer_model_name,
        pad_seq=args.pad_seq,
    )
model = accelerator.prepare(model)
model.device = args.device

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=args.learning_rate,
    betas=(args.beta_1, args.beta_2),
    eps=args.adam_eps,
    weight_decay=args.weight_decay,
)

scheduler = get_linear_warmup_cosine_decay_scheduler(optimizer, args.warmup_steps, args.training_steps, base_lr=args.learning_rate, init_lr=args.init_lr, min_lr=args.learning_rate / args.min_factor, cosine_decay=not args.disable_cosine_decay)
optimizer, scheduler = accelerator.prepare(optimizer, scheduler)

if args.use_wandb:
    accelerator.init_trackers(args.wandb_project, init_kwargs={'wandb': {'name': exp_name, 'config': args}})
else:
    accelerator.init_trackers('')

tasks = [TextTask(args.text_datasets, args.text_datasets_paths, args.sequence_length, tokenizer_model=args.tokenizer_model_name)]
args = args
print_logs = True # args.print_logs
device = torch.device(args.device)

min_lr = args.learning_rate / args.min_factor
deterministic = args.eval_mode == 'deterministic'

exp_name = exp_name
exp_dir = os.path.join(args.save_dir, exp_name)

steps = 0
start_time = None

# Create save dir if does not exist
if args.save_model and not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)

Using pad_token, but it is not set yet.


evaluation/text/loss,▆▅▅▄▄▄▅▄▄▅▃▄▅▆▆▃▅▇▅▅▅▅▅▇▄▁██▆▇█▇▆▆▅▆▄▇▇▆
evaluation/text/perplexity,▂▂▁▁▁▁▁▁▁▁▁▁▁▃▃▁▂▅▂▂▂▂▂▅▁▁▆█▂▃▆▃▂▃▁▂▁▄▃▂
time/evaluation,█▄▅▂▄▅▂▆▅▅▄▃▄▇▃▅▃▄▁▆▃▂▃▅▂▁▅▄▂▅▄▃█▃▅▄▄▃▆▄
time/sample_batch,▃▁▃▁▂▅▁▁▆▃▄▂▄▂▁▄▂▂▂▄▄▃▂▅▅▄▄▄▄▄▂▃▄▃▅▃▄▁▄█
time/total,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
time/training,█▃▂▃▄▂▃▁▂▂▂▂▂▃▂▃▂▂▄▃▃▂▂▃▂▄▃▁▂▂▂▂▂▁▂▃▃▂▂▂
training/learning_rate,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▇▇▇█████████████████████
training/train_loss_mean,██▆▆▅▄▃▃▂▃▂▃▃▂▃▂▃▂▂▂▃▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▁▂▂
training/train_loss_std,▁▁▃▂▂▇█▇▄▂▇▄▃▅▂▅▂▃▂▂▃▆▃▆▇▃▄▃▃▅▅▂▆█▂▄▂█▃▄
evaluation/text/loss,11.33851
evaluation/text/perplexity,83994.42969


/home/bhavul/.conda/envs/neko/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Using pad_token, but it is not set yet.


In [34]:
def sample_text_batch(batch_size):
    batch_dicts = []
    text_tasks = [t for t in tasks if isinstance(t, TextTask)]
    for i,task in enumerate (text_tasks):
        return task.sample_batch(batch_size)

In [35]:
def train_step():
    logs = {}
    logs['training/learning_rate'] = scheduler.get_lr()[0] # store LR at current step
    # Build training batch
    start_time = time.time()

    # Calculate batch size for each task, the following need to be revised to including more new tasks
    text_batch_size = int(args.text_prop * args.batch_size)
    remainder = args.batch_size - text_batch_size

    if remainder > 0: 
        text_batch_size += remainder

    assert args.batch_size == text_batch_size, "Total batch size is not eqaual to the sum of each task's batch size" 

    text_batch_dicts = []

    # Sample text and control batches
    if text_batch_size > 0:
        text_batch_dicts = sample_text_batch(text_batch_size)

    if not text_batch_dicts:  # Handle empty batch case
        # print("Received an empty batch. Skipping this step.")
        return None  # You could return None or handle this case based on your training logic

    # print(f'text_batch_size:{text_batch_size}')

    logs['time/sample_batch'] = time.time() - start_time
    with accelerator.accumulate(model):
        # Compute loss and update model
        logits, loss = model.forward(inputs = text_batch_dicts, compute_loss=True)
        accelerator.backward(loss)

        if not args.disable_grad_clip and accelerator.sync_gradients:
            accelerator.clip_grad_norm_(model.parameters(), args.grad_norm_clip)

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return loss.detach().cpu().item(), logs

In [36]:
def train_iteration(num_steps, iter):
    logs = {}

    train_start = time.time()

    train_losses = []
    steps = 0
    model.train()
    for i in range(num_steps):
        steps += 1
        result = train_step()
        if result is None:
            train_losses.append(train_loss)
            # print("Skipped a training step due to empty batch.")
            continue
        train_loss, step_logs = result
        train_losses.append(train_loss)

    # add logs from last train_step as well
    for log in step_logs:
        logs[log] = step_logs[log]

    logs['time/training'] = time.time() - train_start

    eval_start = time.time()
    model.eval()

    # loop over eval for each env
    with torch.no_grad():
        for task in tasks:
            eval_logs = {}
            if isinstance(task, TextTask):
                eval_logs = task.evaluate(model, num_examples_to_test=args.eval_text_num_examples, deterministic=deterministic, log_examples_to_output=args.eval_text_log_examples)
                for k, v in eval_logs.items():
                    logs[f'evaluation/text/{k}'] = v
                pass

    logs['time/total'] = time.time() - start_time
    logs['time/evaluation'] = time.time() - eval_start
    logs['training/train_loss_mean'] = np.mean(train_losses)
    logs['training/train_loss_std'] = np.std(train_losses)

    if accelerator.is_main_process:
        if print_logs:
            print('=' * 80)
            print(f'Iteration {iter}')
            for k, v in logs.items():
                print(f'{k}: {v}')
            print('=' * 80)

    ## Save model
    if args.save_model and args.save_mode == 'checkpoint':
        accelerator.wait_for_everyone()
        if accelerator.is_main_process:
            unwrapped_model = accelerator.unwrap_model(model)
            save_model(unwrapped_model, exp_dir, f'checkpoint_{steps}', args)
                

    return logs

In [37]:
torch.cuda.empty_cache()

In [38]:
start_time = time.time()
iters = args.training_steps // args.log_eval_freq
print(f'iters:{iters}')
for i in range(iters):
    logs = train_iteration(args.log_eval_freq, i)
    accelerator.log(logs)

## Save model at end of training only if not saving checkpoints
if args.save_model and args.save_mode == 'last':
    accelerator.wait_for_everyone()
    if accelerator.is_main_process:
        unwrapped_model = accelerator.unwrap_model(model)
        save_model(unwrapped_model, exp_dir, f'checkpoint_{steps}', args)
        torch.cuda.empty_cache()    

accelerator.end_training()

iters:1000


/tmp/ipykernel_3289219/2124409890.py:167: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tokens = torch.tensor(batch_dict['text'], dtype=torch.long, device=self.device).unsqueeze(0)


Iteration 0
training/learning_rate: 1.8981999999999999e-06
time/sample_batch: 0.007933378219604492
time/training: 0.6367247104644775
evaluation/text/loss: 0.06441706964233485
evaluation/text/perplexity: 1.0665371417999268
time/total: 6.431450843811035
time/evaluation: 5.794079542160034
training/train_loss_mean: 10.979329204559326
training/train_loss_std: 0.04546026730176001
Iteration 1
training/learning_rate: 3.6964e-06
time/sample_batch: 0.008132696151733398
time/training: 0.5505430698394775
evaluation/text/loss: 0.0992919971616334
evaluation/text/perplexity: 1.10438871383667
time/total: 10.694080591201782
time/evaluation: 3.710369825363159
training/train_loss_mean: 10.903775215148926
training/train_loss_std: 0.0666621446575005
Iteration 2
training/learning_rate: 5.6944e-06
time/sample_batch: 0.007460355758666992
time/training: 0.6261167526245117
evaluation/text/loss: 0.12353754203657268
evaluation/text/perplexity: 1.1314924955368042
time/total: 14.39331841468811
time/evaluation: 3.07

KeyboardInterrupt: 